# Multiscale MILP

This is a continuation of 'One Location, One Temporal Scale, One Operation, Linear Programming Example' [Example 1]. Refer Example 1 to learn the basics on how Energia models processes. 

In this example, we add another Process [Solar PV] and Storage [Li-ion Battery]. Technology choice is modeled using binaries. Moreover, the model is multiscale as the operational capacities are decision variables. 

In [1]:
from energia import *

m = Model('example2')
m.q = Periods()
m.y = 4 * m.q
m.usd = Currency()

## Resources

### Conveniently declaring components

Use m.declare(\<Object Type\>, \<list of names\>) to declare a large number of objects in one step.

In [2]:
m.declare(Resource, ['power', 'wind', 'solar'])

### Set bounds on Resource flows

Unlike wind which has bound on the total consumption, we set a daily limit on solar energy. The same bound is repeated in each quarter. The following constraints are written.

$\mathbf{cons}_{solar, network, quarter_0} \leq 100$

$\mathbf{cons}_{solar, network, quarter_1} \leq 100$

$\mathbf{cons}_{solar, network, quarter_2} \leq 100$

$\mathbf{cons}_{solar, network, quarter_3} \leq 100$

In [3]:
m.solar.consume(m.q) <= 100
m.wind.consume <= 400
m.power.release.prep(180) >= [0.6, 0.7, 0.8, 0.3]

2025-10-23 11:43:31,865 [INFO] Balance for solar in (l0, q): initializing
2025-10-23 11:43:31,866 [INFO] ✔ Completed in 0.0002162456512451172 seconds
2025-10-23 11:43:31,866 [INFO] Binding consume in domain (solar, l0, q)
2025-10-23 11:43:31,867 [INFO] ✔ Completed in 0.0001418590545654297 seconds
2025-10-23 11:43:31,867 [INFO] Balance for wind in (l0, y): initializing
2025-10-23 11:43:31,868 [INFO] ✔ Completed in 9.465217590332031e-05 seconds
2025-10-23 11:43:31,868 [INFO] Binding consume in domain (wind, l0, y)
2025-10-23 11:43:31,868 [INFO] ✔ Completed in 7.2479248046875e-05 seconds
2025-10-23 11:43:31,869 [INFO] Balance for power in (l0, q): initializing
2025-10-23 11:43:31,869 [INFO] ✔ Completed in 0.00012087821960449219 seconds
2025-10-23 11:43:31,869 [INFO] Binding release in domain (power, l0, q)
2025-10-23 11:43:31,870 [INFO] ✔ Completed in 0.00017404556274414062 seconds


## Operations 

### Capacity as a variable 

Here we want the optimization problem to determine the optimal capacity. Moreover, we set binaries to avoid the lower bound being adhered to if the process is not set up. 

If the bounds are meant to be compulsory limits, skip the .x 

In [4]:
m.wf = Process()
m.wf(m.power) == -1 * m.wind
m.wf.capacity.x <= 100
m.wf.capacity.x >= 10
m.capacity.show()

2025-10-23 11:43:31,875 [INFO] Binding capacity in domain (wf, l0, y)
2025-10-23 11:43:31,875 [INFO] ✔ Completed in 0.00017595291137695312 seconds
2025-10-23 11:43:31,876 [INFO] Binding capacity in domain (wf, l0, y)
2025-10-23 11:43:31,876 [INFO] ✔ Completed in 0.00016450881958007812 seconds


<IPython.core.display.Math object>

<IPython.core.display.Math object>

Unlike in Example 1, where the capacity was know, capacity is a variable here. 

Moreover, the expenditure associated with operating and capacitating are different

In [5]:
m.wf.operate.prep(norm=True) <= [0.9, 0.8, 0.5, 0.7]
m.wf.capacity[m.usd.spend] == 990637 + 3354
m.wf.operate[m.usd.spend] == 49
m.operate.show(True)

2025-10-23 11:43:31,884 [INFO] Binding operate in domain (wf, l0, q)
2025-10-23 11:43:31,885 [INFO] ✔ Completed in 0.0002391338348388672 seconds
2025-10-23 11:43:31,886 [INFO] Mapping operate: (wf, l0, q) → (wf, l0, y)
2025-10-23 11:43:31,886 [INFO] ✔ Completed in 0.00011682510375976562 seconds


<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [6]:
m.pv = Process()
m.pv(m.power) == -1 * m.solar
m.pv.capacity.x <= 100
m.pv.capacity.x >= 10

m.pv.operate.prep(norm=True) <= [0.6, 0.8, 0.9, 0.7]
m.pv.capacity[m.usd.spend] == 567000 + 872046
m.pv.operate[m.usd.spend] == 90000

2025-10-23 11:43:31,895 [INFO] Binding capacity in domain (pv, l0, y)
2025-10-23 11:43:31,895 [INFO] ✔ Completed in 0.0001842975616455078 seconds
2025-10-23 11:43:31,896 [INFO] Binding capacity in domain (pv, l0, y)
2025-10-23 11:43:31,896 [INFO] ✔ Completed in 0.00016808509826660156 seconds
2025-10-23 11:43:31,897 [INFO] Binding operate in domain (pv, l0, q)
2025-10-23 11:43:31,897 [INFO] ✔ Completed in 0.00019240379333496094 seconds
2025-10-23 11:43:31,898 [INFO] Mapping operate: (pv, l0, q) → (pv, l0, y)
2025-10-23 11:43:31,899 [INFO] ✔ Completed in 0.00010967254638671875 seconds


### Storage Operation

energia now allows storing to require the use of other resources, example power for hydrogen cryogenic storage. 

Provide an equation similar to Process, in this case the basis is the stored resource 
If no other resource is provided, it is assumed to be the charging/discharging efficiency

Note that the following are created internally: 
1. auxilary resource  with name resource.stored 
2. charging and discharging processes as storage.charge and storage.discharge 

The parameters for each of these can be set individually, thus allowing for a wide range of modeling approaches 

In [7]:
m.lii = Storage()

m.lii(m.power) == 0.9
m.lii.capacity.x <= 100
m.lii.capacity.x >= 10

# m.lii.capacity >= 10
m.lii.capacity[m.usd.spend] == 1302182 + 41432

m.lii.inventory[m.usd.spend] == 2000
m.lii.charge.capacity <= 100
m.lii.discharge.capacity <= 100

2025-10-23 11:43:31,904 [INFO] Binding invcapacity in domain (lii.stored, l0, y)
2025-10-23 11:43:31,905 [INFO] ✔ Completed in 0.00018715858459472656 seconds
2025-10-23 11:43:31,905 [INFO] Binding invcapacity in domain (lii.stored, l0, y)
2025-10-23 11:43:31,906 [INFO] ✔ Completed in 0.00014591217041015625 seconds
2025-10-23 11:43:31,907 [INFO] Balance for lii.stored in (l0, y): initializing
2025-10-23 11:43:31,907 [INFO] Binding capacity in domain (lii.charge, l0, y)
2025-10-23 11:43:31,908 [INFO] ✔ Completed in 7.462501525878906e-05 seconds
2025-10-23 11:43:31,908 [INFO] Binding capacity in domain (lii.discharge, l0, y)
2025-10-23 11:43:31,909 [INFO] ✔ Completed in 6.4849853515625e-05 seconds


## Locating Operations

Operations can be located as 

operation.locate(\<list of locations\>)

or 

m.location.operations(\<list of operations\>)

They both do the same thing 

In [8]:
m.pem = Process()

m.pem(-m.power) == m.solar 

η(pem, power)

In [9]:
m.pv.locate(m.network)
m.network.locate(m.wf, m.lii)

2025-10-23 11:43:31,918 [INFO] Balance for power in (l0, q): adding produce(power, l0, q, operate, pv)
2025-10-23 11:43:31,919 [INFO] ✔ Completed in 0.0001647472381591797 seconds
2025-10-23 11:43:31,920 [INFO] Balance for solar in (l0, q): adding expend(solar, l0, q, operate, pv)
2025-10-23 11:43:31,921 [INFO] ✔ Completed in 0.00014090538024902344 seconds
2025-10-23 11:43:31,922 [INFO] Balance for power in (l0, q): adding produce(power, l0, q, operate, wf)
2025-10-23 11:43:31,922 [INFO] ✔ Completed in 0.0001461505889892578 seconds
2025-10-23 11:43:31,923 [INFO] Balance for wind in (l0, y): adding expend(wind, l0, y, operate, wf)
2025-10-23 11:43:31,923 [INFO] ✔ Completed in 0.000102996826171875 seconds
2025-10-23 11:43:31,924 [INFO] Assuming  lii.stored inventory capacity is unbounded in (l0, y)
2025-10-23 11:43:31,924 [INFO] Assuming inventory of lii.stored is unbounded in (l0, y)
2025-10-23 11:43:31,924 [INFO] Assuming inventory of lii.stored is bound by inventory capacity in (l0, q)

## Inventory Balance

Inventory is passed on from one time period (t - 1) to the next (t) and hence features in the general resource balance for resource.stored 

In [10]:
m.inventory.show()

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

## Optimize!

In [11]:
m.usd.spend.opt()

2025-10-23 11:43:31,946 [INFO] Mapping spend: (usd, l0, y, capacity, wf) → (usd, l0, y)
2025-10-23 11:43:31,947 [INFO] ✔ Completed in 0.0001239776611328125 seconds
2025-10-23 11:43:31,947 [INFO] Mapping spend: (usd, l0, y, operate, wf) → (usd, l0, y)
2025-10-23 11:43:31,948 [INFO] ✔ Completed in 0.00012302398681640625 seconds
2025-10-23 11:43:31,948 [INFO] Mapping spend: (usd, l0, y, capacity, pv) → (usd, l0, y)
2025-10-23 11:43:31,949 [INFO] ✔ Completed in 0.00012183189392089844 seconds
2025-10-23 11:43:31,949 [INFO] Mapping spend: (usd, l0, y, operate, pv) → (usd, l0, y)
2025-10-23 11:43:31,950 [INFO] ✔ Completed in 0.00015592575073242188 seconds
2025-10-23 11:43:31,950 [INFO] Mapping spend: (usd, l0, y, invcapacity, lii.stored) → (usd, l0, y)
2025-10-23 11:43:31,950 [INFO] ✔ Completed in 0.00011348724365234375 seconds
2025-10-23 11:43:31,951 [INFO] Mapping spend: (usd, l0, y, inventory, lii.stored) → (usd, l0, y)
2025-10-23 11:43:31,951 [INFO] ✔ Completed in 0.00015211105346679688 s

Set parameter Username
Academic license - for non-commercial use only - expires 2026-08-01
Read MPS format model from file Program(example2).mps
Reading time = 0.00 seconds
PROGRAM(EXAMPLE2): 85 rows, 78 columns, 198 nonzeros


2025-10-23 11:43:31,961 [INFO] Optimizing Program(example2) using gurobi


Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (win64 - Windows 11.0 (26100.2))

CPU model: 13th Gen Intel(R) Core(TM) i7-13700, instruction set [SSE2|AVX|AVX2]
Thread count: 16 physical cores, 24 logical processors, using up to 24 threads

Optimize a model with 85 rows, 78 columns and 198 nonzeros
Model fingerprint: 0x2d658939
Variable types: 75 continuous, 3 integer (3 binary)
Coefficient statistics:
  Matrix range     [6e-01, 1e+06]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [7e+01, 4e+02]
Presolve removed 70 rows and 63 columns
Presolve time: 0.00s
Presolved: 15 rows, 15 columns, 44 nonzeros
Variable types: 15 continuous, 0 integer (0 binary)

Root relaxation: objective 3.006497e+08, 4 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0    3.006497e+08 3.0065e+08  0.0

2025-10-23 11:43:31,970 [INFO] Solution found. Use .output() to display it
2025-10-23 11:43:31,971 [INFO] Creating Solution object, check.solution


## Solution

### Inventory Profiles

The inventory maintained in each time period is:

In [12]:
m.inventory.output()

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

The amount charged into inventory is:

In [13]:
m.produce(m.power.lii, m.lii.charge.operate, m.q).output()

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

The amount discharged from inventory is:

In [14]:
m.produce(m.power, m.lii.discharge.operate, m.q).output()

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

### Integer Decisions 

All the operations are setup in this case

In [15]:
m.capacity.reporting.output()

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [16]:
m.capacity.output()

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [17]:
m.show(True)

# Mathematical Program for Program(example2)

<br><br>

## Index Sets

<IPython.core.display.Math object>

<IPython.core.display.Math object>

...

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [19]:
m.power.lii 

lii.stored

In [20]:
m.power.wf

wf

In [22]:
m.power.lii

lii.stored

In [23]:
m.lii.stored

lii.stored